In [1]:
import os
import re
import sys
from collections import Counter
from urllib.parse import urlparse, urlsplit, parse_qs, parse_qsl

import numpy as np
import parsel
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_classification_report, sequence_accuracy_score
from sklearn.model_selection import cross_val_predict, GroupKFold

sys.path.insert(0, '..')
from autopager.storage import Storage
from autopager.htmlutils import (get_link_text, get_text_around_selector_list, 
                                 get_link_href, get_selector_root)
from autopager.utils import (
    get_domain, normalize_whitespaces, normalize, ngrams, tokenize, ngrams_wb, replace_digits
)
from autopager.model import link_to_features, _num_tokens_feature, _elem_attr

In [2]:
storage = Storage()
urls = [rec['Page URL'] for rec in storage.iter_records()]
X_raw, y = storage.get_Xy()
print("pages: {}  domains: {}".format(len(urls), len({get_domain(url) for url in urls})))

Not all links are matched {'<a>1</a>'}
Not all links are matched {'<a>6</a>'}
Not all links are matched {'<a onclick="display(\'grid\');">Mosaico</a>'}
Not all links are matched {'<a onclick="display(\'grid\');">Mosaico</a>'}
Not all links are matched {'<a class="btn prev disabled" type="backward"><i class="icon-chevron-left"> </i></a>'}
pages: 196  domains: 81


In [3]:
%%time
# XXX: these functions should be copy-pasted from autopager/model.py

def link_to_features(link):
    text = normalize(get_link_text(link))

    href = get_link_href(link)
    p = urlsplit(href)

    query_parsed = parse_qsl(p.query)
    query_param_names = [k.lower() for k, v in query_parsed]
    query_param_names_ngrams = ngrams_wb(
        " ".join([normalize(name) for name in query_param_names]), 3, 5, True
    )

    elem = get_selector_root(link)
    elem_target = _elem_attr(elem, 'target')
    elem_rel = _elem_attr(elem, 'rel')

    # Classes of link itself and all its children.
    # It is common to have e.g. span elements with fontawesome
    # arrow icon classes inside <a> links.
    self_and_children_classes = ' '.join(link.xpath(".//@class").extract())
    parent_classes = ' '.join(link.xpath('../@class').extract())
    css_classes = normalize(self_and_children_classes + ' ' + parent_classes)

    return {
        'bias': 3.0,
        'isdigit': text.isdigit(),
        'isalpha': text.isalpha(),
        'elem-target': elem_target,
        'elem-rel': elem_rel,
        'num-tokens%s' % _num_tokens_feature(text): 1.0,

        'text': ngrams_wb(replace_digits(text), 2, 5),
        'text-exact': replace_digits(text.strip()[:20].strip()),
        'class': ngrams_wb(css_classes, 4, 5),
        'query': query_param_names_ngrams,

        'path-has-page': 'page' in p.path.lower(),
        'path-has-pageXX': re.search(r'[/-](?:p|page\w?)/?\d+', p.path.lower()) is not None,
        'path-has-number': any(part.isdigit() for part in p.path.split('/')),

        'href-has-year': re.search('20\d\d', href) is not None,
    }


def page_to_features(xseq):
    features = [link_to_features(a) for a in xseq]
    around = get_text_around_selector_list(xseq, max_length=15)

    # weight is less than 1 because there is a lot of duplicate information
    # in these ngrams and so we want to regularize them stronger
    # (as if they are a single feature, not many features)
    k = 0.2
    for feat, (before, after) in zip(features, around):
        feat['text-before'] = {n: k for n in ngrams_wb(normalize(before), 5, 5)}
        feat['text-after'] = {n: k for n in ngrams_wb(normalize(after), 5, 5)}
    return features


X = [page_to_features(xseq) for xseq in X_raw]

CPU times: user 13 s, sys: 284 ms, total: 13.3 s
Wall time: 13.5 s


In [4]:
# X[60][12]

In [5]:
# TRAIN_SIZE = 80
# X_train, y_train = X[:TRAIN_SIZE], y[:TRAIN_SIZE]
# X_test, y_test = X[TRAIN_SIZE:], y[TRAIN_SIZE:]
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.001, 
    c2=0.05, 
    max_iterations=100, 
    all_possible_transitions=True,
    verbose=False,
)
# crf.fit(X_train, y_train, X_test, y_test)

We must be careful when splitting the dataset into training and
evaluation parts: pages from the same domain should be in the same
"bin". There could be several pages from the same domain, and these
pages may have duplicate or similar link patterns
(e.g. a particular CSS class for paginator links). If we put one such page in a training dataset and another in
an evaluation dataset then the metrics will be too optimistic,
and they can make us to choose wrong features/models.

In [6]:
%%time
group_kfold = GroupKFold(n_splits=6)
groups = [get_domain(url) for url in urls]
y_pred = cross_val_predict(crf, X, y, cv=group_kfold, groups=groups, n_jobs=-1)
print(flat_classification_report(y, y_pred, labels=['PAGE', 'NEXT', 'PREV'], digits=3))
print("Sequence accuracy: {:0.3f}".format(sequence_accuracy_score(y, y_pred)))

             precision    recall  f1-score   support

       PAGE      0.871     0.924     0.897      1178
       NEXT      0.881     0.768     0.821       155
       PREV      0.897     0.777     0.833       112

avg / total      0.874     0.896     0.884      1445

Sequence accuracy: 0.582
CPU times: user 9.06 s, sys: 1.66 s, total: 10.7 s
Wall time: 1min 9s


In [7]:
crf.fit(X, y)
# crf.attributes_
crf.num_attributes_

11675

In [8]:
# [a for a in sorted(crf.attributes_) if a.startswith('id')]

## What are important features?

In [9]:
from eli5 import explain_weights

In [10]:
# XXX: weight for correlated features don't show their importance
# XXX: weights for features of different scale don't show their importance
# (e.g. coefficients to text-after and text-before features are high, but only 
# because input is scaled down for these features)

explain_weights(crf, top=50)

Explanation(estimator="CRF(algorithm='lbfgs', all_possible_states=None,\n  all_possible_transitions=True, averaging=None, c=None, c1=0.001, c2=0.05,\n  calibration_candidates=None, calibration_eta=None,\n  calibration_max_trials=None, calibration_rate=None,\n  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,\n  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,\n  max_linesearch=None, min_freq=None, model_filename=None,\n  num_memories=None, pa_type=None, period=None, trainer_cls=None,\n  variance=None, verbose=False)", description=None, error=None, method='CRF', is_regression=False, targets=[TargetExplanation(target='O', feature_weights=FeatureWeights(pos=[('elem-rel:nofollow', 2.1577280000000001), ('text:..', 1.6734819999999999), ('text-exact:>>', 1.4617880000000001), ('num-tokens>2', 1.4019550000000001), ('text-after:...', 1.3890899999999999), ('href-has-year', 1.347078), ('query:m', 1.3271539999999999), ('text-before:下部', 1.1916739999999999), ('text-after:seite', 1.1433960000000001), ('text-before:ージ送り', 1.0599769999999999), ('text-after:eiten', 1.035128), ('text-after:iten:', 1.0338799999999999), ('text-after:平台业务', 1.02007), ('bias', 1.011498), ('text-after:start', 1.0106090000000001), ('text-after:per', 1.010078), ('class:ing-n', 0.99712400000000001), ('class:g-ne', 0.99712400000000001), ('class:ng-n', 0.99712400000000001), ('class:ng-ne', 0.99712400000000001), ('class:g-nex', 0.99712400000000001), ('text-after:page', 0.99551699999999999), ('text-before:はじまり', 0.99036400000000002), ('elem-target:_blank', 0.96480699999999997), ('num-tokens=2', 0.95349799999999996), ('class:day', 0.94662800000000002), ('text:...', 0.84035400000000005)], neg=[('text:首页', -1.895826), ('path-has-pageXX', -1.7864789999999999), ('query:page', -1.5946979999999999), ('query:age', -1.362053), ('path-has-page', -1.2165809999999999), ('text-before:page', -1.21584), ('text-after::', -1.2068220000000001), ('text-after:به', -1.1856580000000001), ('text-before:•', -1.1597489999999999), ('query:pag', -1.1318710000000001), ('text-after:threa', -1.12199), ('text-before:pages', -1.118779), ('text:>', -1.0381069999999999), ('text-before:per', -1.02685), ('class:page', -0.98400500000000002), ('query:art', -0.89145600000000003), ('query:start', -0.887845), ('query:tart', -0.887845), ('class:prev', -0.870363), ('query:tar', -0.863931), ('query:sta', -0.86194899999999997), ('text-before:pm', -0.84240700000000002), ('query:star', -0.834314)], pos_remaining=9440, neg_remaining=1002), proba=None, score=None, weighted_spans=None), TargetExplanation(target='PAGE', feature_weights=FeatureWeights(pos=[('isdigit', 3.0836100000000002), ('text:末页', 2.9788589999999999), ('text-exact:末页', 2.9788589999999999), ('text-exact:首页', 1.9632229999999999), ('text:首页', 1.852973), ('text-exact:X', 1.668784), ('text:<<', 1.546413), ('text:X', 1.410045), ('text-after:oninf', 1.3062849999999999), ('text-after:iconi', 1.3062849999999999), ('text-after:conin', 1.3062849999999999), ('text-after:ninfo', 1.3062849999999999), ('class:last', 1.2582310000000001), ('text-before:page', 1.217991), ('elem-rel:next', 1.2106300000000001), ('text-after::', 1.2017370000000001), ('text-after:رفتن', 1.200232), ('text-after:صفحه', 1.200232), ('text-after:به', 1.1962060000000001), ('text-before:•', 1.1789229999999999), ('text-before:pages', 1.167788), ('text:|<', 1.134368), ('text-after:threa', 1.1294439999999999), ('text-before:per', 1.106298), ('text-exact:<<', 1.052959), ('text:>|', 1.0087060000000001), ('text-exact:XX', 1.0080089999999999), ('text-before:pm', 0.86420300000000005), ('text-before:2', 0.83762099999999995), ('text-before:1', 0.813666), ('text:XX', 0.80432000000000003), ('text-after:itgli', 0.79895300000000002), ('text-after:mitgl', 0.79895300000000002), ('text-after:glied', 0.79895300000000002), ('text-after:aktiv', 0.79895300000000002), ('text-after:tglie', 0.79895300000000002), ('text-after:ktive', 0.79895300000000002), ('class:firs', 0.7939589

## Let's check errors the model is making

In [11]:
group_kfold = GroupKFold(n_splits=6)
groups = [get_domain(url) for url in urls]
y_pred = cross_val_predict(crf, X, y, cv=group_kfold, groups=groups, n_jobs=-1)

errors = np.asarray(y) != np.asarray(y_pred)
error_rows = np.asarray(list(storage.iter_records()))[errors]
error_links = np.asarray(X_raw)[errors]
error_y_pred = y_pred[errors]
error_y_true = np.asarray(y)[errors]

In [12]:
for links, yseq_pred, yseq_true, row in zip(error_links, error_y_pred, error_y_true, error_rows):
    print(row['Page URL'])
    for label_correct, label_pred, link in zip(yseq_true, yseq_pred, links.extract()):
        if label_correct != label_pred:
            print("%4s %4s %s" % (label_correct, label_pred, link))
    print("\n")

https://www.mypapershop.com/patricks-day-supplies.html
NEXT    O <a href="https://www.mypapershop.com/mm5/merchant.mvc?Session_ID=56e7cf6116038eec3b953a976519a103&amp;Store_Code=MPS&amp;Screen=CTGY&amp;Category_Code=patricks-day-supplies&amp;CatListingOffset=24&amp;Offset=24&amp;Per_Page=24&amp;Sort_By=disp_order" class="searchspring-next">▶</a>
NEXT    O <a href="https://www.mypapershop.com/mm5/merchant.mvc?Session_ID=56e7cf6116038eec3b953a976519a103&amp;Store_Code=MPS&amp;Screen=CTGY&amp;Category_Code=patricks-day-supplies&amp;CatListingOffset=24&amp;Offset=24&amp;Per_Page=24&amp;Sort_By=disp_order" class="searchspring-next">▶</a>


http://www.newschittagong24.com/?cat=1
PAGE NEXT <a href="http://www.newschittagong24.com/?cat=1&amp;paged=1422" class="last" title="শেষ »">শেষ »</a>


http://www.newschittagong24.com/?cat=1&paged=5
PAGE    O <a href="http://www.newschittagong24.com/?cat=1" class="first" title="« প্রথম">« প্রথম</a>
PAGE NEXT <a href="http://www.newschittagong24.com/?cat=1

## Unused code

In [13]:
def _url_parts(url):
    p = urlsplit(url)
    args = parse_qsl(p.query)
    argnames = [name for name, value in args]
    return p.netloc, set(p.path.split('/')) | set(args) | set(argnames)

def url_distance(url1, url2):        
    netloc1, parts1 = _url_parts(url1)
    netloc2, parts2 = _url_parts(url2)
    if netloc1 != netloc2:
        return 1.0
    if not parts1 and not parts2:
        return 0.0
    return 1 - len(parts1 & parts2) / len(parts1 | parts2)

#         dist = url_distance(url, href)
#         if dist == 0:
#             feat['url-distance=0'] = 1.0
#         elif dist == 1.0:
#             feat['url-distance=1'] = 1.0
#         else:
#             feat['url-distance=k'] = dist


url_distance('http://example.com/foo/345?page=2', 'http://example.com/foo/345?page=4')

0.33333333333333337

In [14]:
# def guess_page_number(link):
#     text = get_link_text(link).strip()
#     if text.isdigit():
#         return int(text)
#     return None
    
# def number_pattern2(pattern):
#     txt = re.sub('X+', 'X', pattern)
# #     txt = re.sub('C+', 'C', txt)
#     return txt
    
#     pagenums = [guess_page_number(a) for a in xseq]
# #     print(pagenums)
#     for i in range(1, len(xseq)):
#         if pagenums[i-1] is None or pagenums[i] is None:
#             features[i]['page-diff:None'] = 1.0
#         else:
#             diff = pagenums[i] - pagenums[i-1]
#             if diff == 1:
#                 features[i]['page-diff==1'] = 1.0
#             else:
#                 features[i]['page-diff<>1'] = 1.0